In [ ]:
import threading
from http.server import BaseHTTPRequestHandler, HTTPServer

from opentelemetry.proto.collector.trace.v1.trace_service_pb2 import ExportTraceServiceRequest


class OTLPCollector(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)

        traces_request = ExportTraceServiceRequest()
        traces_request.ParseFromString(post_data)
        
        print("Received a POST request with data:")
        print(traces_request)

        self.send_response(200, "Traces received")
        self.end_headers()
        self.wfile.write(b'Data received and printed to stdout.\n')

def run_server(port: int):
    server_address = ('', port)
    httpd = HTTPServer(server_address, OTLPCollector)
    httpd.serve_forever()

def start_server(port: int):
    server_thread = threading.Thread(target=run_server, args=(port,))
    server_thread.daemon = True
    server_thread.start()
    print(f"Server started on port {port}. Access http://localhost:{port}/")
    return server_thread

In [ ]:
start_server(port=4318)

In [ ]:
import os

from promptflow.tracing import start_trace

os.environ["PF_TRACING_SKIP_LOCAL_SETUP"] = "true"
start_trace()

In [ ]:
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor

tracer_provider = trace.get_tracer_provider()
otlp_span_exporter = OTLPSpanExporter()
tracer_provider.add_span_processor(BatchSpanProcessor(otlp_span_exporter))

In [ ]:
from llm import my_llm_tool

deployment_name = "text-davinci-003"

In [ ]:
result = my_llm_tool(
    prompt="Write a simple Hello, world! program that displays the greeting message when executed.",
    deployment_name=deployment_name,
)
result
# view the traces under this cell